Getting Dataset Ready

In [ ]:
!pip install datasets ciso8601

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# IMPORTS
from datasets import load_dataset
from datasets.iterable_dataset import IterableDataset
import requests
import ciso8601
import time
import json
from datetime import datetime

DATASET_URI = "codeparrot/github-code"


class CodeSnippetDataset:
    def construct_feature_set(self, code_entry):
        features = dict();
        user_repo_name = code_entry['repo_name'].split('/')
        owner = user_repo_name[0]
        repo = "".join(user_repo_name[1:])
        repo_response = requests.get(f"https://api.github.com/repos/{owner}/{repo}")
        content = json.loads(repo_response.text)
        features["num_stars"] = content.get("stargazers_count", 0)
        features["num_forks"] = content.get("forks_count", 0)
        features["num_watchers"] = content.get("watchers_count", 0)
        features["num_open_issues"] = content.get("open_issues_count", 0)
        parsed_datetime = ciso8601.parse_datetime(content.get("created_at", datetime.now().isoformat()))
        timestamp = time.mktime(parsed_datetime.timetuple())
        features["created_at"] = timestamp
        return features
    # THE BELOW IS FOR PYTHON FILES (LEVERAGES INDENTATION RULES)
    def construct_list_of_functions(self, code_entry):
        raw_code = code_entry["code"]
        lines = raw_code.split('\n')
        start = -1
        functions = []
        amnt_tabs = 0
        for i in range(len(lines)):
            # disregard empty lines (prune trailing whitespace later)
            if (start != -1 and len(lines[i]) > 0):
                amnt_tabs_new = len(lines[i].rstrip()) - len(lines[i].strip())
                if amnt_tabs_new <= amnt_tabs:
                    functions.append(("\n".join(lines[start:i])).strip())
                    start = -1
            elif lines[i].lstrip().startswith("def "):
                start = i
                amnt_tabs = len(lines[i].rstrip()) - len(lines[i].strip())
        return functions
     
    def augment_code_entry(self, entry):
        entry["reputation_features"] = self.construct_feature_set(entry)
        entry["functions"] = self.construct_list_of_functions(entry)
        return entry
        
    def get_n_snippets(self, n):
        """Provides the next n code snippets from the GitHub dataset. 

        Args:
            n (int): number of snippets desired

        Returns:
            list[dict]: an array of size n, where each element is a dictionary containing the functions / 
            reputation features of a code snippet.
        """
        snippets = self.dataset.take(n).remove_columns("code")
        self.dataset = self.dataset.skip(n)
        return list(snippets)        

    def __init__(self, languages):
        """__init__
        Args:
            languages (list[str]): programming languages to be included. Must be some of "Python", "Javascript", "Java", etc...
                                   for now only supports Python
        """
        
        self.dataset : IterableDataset = load_dataset(DATASET_URI, streaming=True, split='train', languages=languages)
        self.dataset : IterableDataset = self.dataset.map(self.augment_code_entry)
        

Code Documentation Generation

In [ ]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

# Large Models
large_multitask_finetune = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask_finetune"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask_finetune", skip_special_tokens=True),
    device=0
)
# large_multitask = SummarizationPipeline(
#     model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask"),
#     tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask", skip_special_tokens=True),
#     device=0
# )
# large_transfer_learning_finetune = SummarizationPipeline(
#     model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_python_transfer_learning_finetune"),
#     tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_python_transfer_learning_finetune", skip_special_tokens=True),
#     device=0
# )


# Base Models
# base_multitask_finetune = SummarizationPipeline(
#     model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_base_code_documentation_generation_python_multitask_finetune"),
#     tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_base_code_documentation_generation_python_multitask_finetune", skip_special_tokens=True),
#     device=0
# )
# base_multitask = SummarizationPipeline(
#     model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_base_code_documentation_generation_python_multitask"),
#     tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_base_code_documentation_generation_python_multitask", skip_special_tokens=True),
#     device=0
# )
# base = SummarizationPipeline(
#     model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_base_code_documentation_generation_python"),
#     tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_base_code_documentation_generation_python", skip_special_tokens=True),
#     device=0
# )
# base_transfer_learning_finetune = SummarizationPipeline(
#     model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_base_code_documentation_generation_python_transfer_learning_finetune"),
#     tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_base_code_documentation_generation_python_transfer_learning_finetune", skip_special_tokens=True),
#     device=0
# )

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1248: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Process Data

In [ ]:
def process_data(snippets):
  code_reference = {}
  function_ids = []

  count = 0
  for snippet in snippets:
    for i, func in enumerate(snippet['functions']):
      id = snippet['repo_name']+"_"+snippet['path']+"_"+str(i)
      
      function_ids.append(id)
      code_reference[id] = {
          "code": func,
          "documentation": large_multitask_finetune([func])[0]['summary_text'],
          "reputation": snippet['reputation_features']
      }
      count += 1

    print(str(count) + " functions processed.")
  
  print("No. of processed functions: ", len(code_reference.keys()))
  return [code_reference, function_ids]

In [ ]:
# Get dataset
ds = CodeSnippetDataset(languages=["Python"])
snippets = ds.get_n_snippets(1000);

In [ ]:
output = process_data(snippets)

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Your max_length is set to 20, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 20, but you input_length is only 14. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 20, but you input_length is only 16. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


15 functions processed.
28 functions processed.


Your max_length is set to 20, but you input_length is only 17. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 20, but you input_length is only 16. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 20, but you input_length is only 13. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
Your max_length is set to 20, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 20, but you input_length is only 11. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)
Your max_length is set to 20, but you input_length is only 12. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
Your max_length is set to 20, but you input_length is only 12. You might consider decreasing m

66 functions processed.
68 functions processed.
72 functions processed.
72 functions processed.
72 functions processed.


Your max_length is set to 20, but you input_length is only 14. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 20, but you input_length is only 17. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 20, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


...

5958 functions processed.


Your max_length is set to 20, but you input_length is only 18. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


5975 functions processed.


Your max_length is set to 20, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 20, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 20, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 20, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 20, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 20, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 20, but you input_length is only 13. You might consider decreasing m

6550 functions processed.
6550 functions processed.
No. of processed functions:  6550


In [ ]:
with open("output.json", "w") as f:
    json.dump({
      "function_ids": output[1],
      "code_reference": output[0]
    }, f)